# HWRS 528 IMP04
## Optimization of performance by automated calibration of parameters
### 1. Import Python packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import LRC_modelSubprograms as lrc      # Leaf River Catchment model subprograms
from scipy.optimize import minimize     # Optimization package for par optimization